In [ ]:
import pandas as pd

# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_excel('./analysis_results_all.xlsx', sheet_name='data')
df.describe()


/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,p1_k2,pis1_tran2,iav,dav,sv,lcm_adj,lcm_all,spos,sneg,generalizations,...,ppron12,pcon,adja,interj,aglt,siebie,adjp,bedzie,date,tokens
count,1497.000000,1497.000000,1497.000000,1497.000000,1497.000000,1497.000000,1497.000000,1497.000000,1497.000000,1497.000000,...,1497.000000,1497.000000,1497.000000,1497.000000,1497.000000,1496.000000,1497.000000,1497.000000,1497.000000,1497.000000
mean,1.498998,1.492318,60.731463,62.617234,99.846359,1.896460,3106.496326,10.825651,5.197729,1.002004,...,1.695391,0.705411,0.374749,3.468270,1.191049,0.701203,0.841015,0.517034,0.071476,224.015364
std,0.500166,0.500108,110.738606,138.489362,173.755004,2.119894,12658.520124,8.050953,4.683675,1.528472,...,2.641592,1.015615,0.964187,5.438692,2.113643,1.271093,1.935484,1.241499,0.320169,163.266916
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.000000
25%,1.000000,1.000000,15.000000,12.000000,23.000000,0.000000,325.000000,5.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,118.000000
50%,1.000000,1.000000,32.000000,26.000000,49.000000,1.000000,816.000000,9.000000,4.000000,0.000000,...,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,173.000000
75%,2.000000,2.000000,65.000000,65.000000,110.000000,3.000000,2272.000000,14.000000,7.000000,1.000000,...,2.000000,1.000000,0.000000,5.000000,2.000000,1.000000,1.000000,1.000000,0.000000,280.000000
max,2.000000,2.000000,2399.000000,2507.000000,3150.000000,16.000000,334267.000000,56.000000,41.000000,11.000000,...,22.000000,7.000000,9.000000,47.000000,25.000000,16.000000,20.000000,13.000000,4.000000,1371.000000


In [ ]:
df = df[df["pis1_tran2"] == 1.0] 

In [ ]:
feature_cols = list(df)[3:]
print(len(feature_cols))
feature_cols

43


['iav',
 'dav',
 'sv',
 'lcm_adj',
 'lcm_all',
 'spos',
 'sneg',
 'generalizations',
 'negations',
 'c_mhd2',
 'c_mdd2',
 'c_gun_fog_cnt',
 'placeName',
 'persName',
 'geogName',
 'orgName',
 'prep',
 'subst',
 'conj',
 'fin',
 'adj',
 'pact',
 'praet',
 'adv',
 'num',
 'comp',
 'pred',
 'inf',
 'ppas',
 'liczba_zdan',
 'ppron3',
 'chars',
 'winien',
 'ppron12',
 'pcon',
 'adja',
 'interj',
 'aglt',
 'siebie',
 'adjp',
 'bedzie',
 'date',
 'tokens']

In [ ]:
X = df.loc[:, feature_cols].fillna(0)
X.shape

(760, 43)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVC, LinearSVC, SVC
from xgboost import XGBClassifier
import numpy as np
import warnings


Y = np.array(y)
X = np.array(X)

split_nr = 0
loo = KFold(n_splits=20)
all_predicted_y = []
for train_index, test_index in loo.split(Y):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)


    #clf = SVC(kernel='rbf')
    #clf = NuSVC(nu=n, kernel=kernel_type)
    #clf = LinearSVC(max_iter=100000)
    clf = XGBClassifier()
    model = clf.fit(X_train, Y_train.ravel())
    y_pred = model.predict(X_test)
    all_predicted_y.extend( y_pred )
    print(split_nr)
    print(accuracy_score(Y_test, y_pred))
    split_nr+=1

acc = accuracy_score(Y, all_predicted_y)
std_error = np.std( np.abs(np.subtract(Y, all_predicted_y)) ) / np.sqrt( len(all_predicted_y) )
print("accuracy: %1.4f std_error: %1.4f" % (acc, std_error))

0
0.631578947368421
1
0.631578947368421
2
0.5526315789473685
3
0.5789473684210527
4
0.631578947368421
5
0.42105263157894735
6
0.5263157894736842
7
0.4473684210526316
8
0.5789473684210527
9
0.5
10
0.5
11
0.47368421052631576
12
0.5263157894736842
13
0.47368421052631576
14
0.5263157894736842
15
0.5263157894736842
16
0.5526315789473685
17
0.5526315789473685
18
0.5
19
0.5
accuracy: 0.5316 std_error: 0.0181
